<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster.  
- You'll start with creating a text file format, since that's the type of data you are reading in. 
- Next, you'll create a data source for the SQL Storage Pool, since that allows you to access the HDFS system in BDC. 
- Finally, you'll create an External Table, which uses the previous steps to access the data.


In [1]:
/* Create External File Format */

USE sales;
GO

IF NOT EXISTS
(
    SELECT 1
    FROM sys.external_file_formats
    WHERE
        name = 'csv_file'
)
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH
    (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS
        (
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE
        )
    );
END

In [3]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS
(
     SELECT 1
     FROM sys.external_data_sources
     WHERE
          name = 'SqlStoragePool'
)
BEGIN
     CREATE EXTERNAL DATA SOURCE SqlStoragePool
     WITH (LOCATION = 'sqlhdfs://controller-svc:8080/default');
END

In [4]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS
(
    SELECT 1
    FROM sys.external_tables
    WHERE
        name = 'partner_customers_hdfs'
)
BEGIN
    CREATE EXTERNAL TABLE dbo.partner_customers_hdfs
    (
        CustomerSource VARCHAR(250),
        CustomerName VARCHAR(250),
        EmailAddress VARCHAR(250)
    )
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
END

In [17]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP(100)
    hdfs.CustomerSource,
    hdfs.CustomerName,
    hdfs.EmailAddress
FROM dbo.partner_customers_hdfs hdfs
WHERE
    hdfs.EmailAddress LIKE '%wingtip%'
ORDER BY
    CustomerSource,
    CustomerName;
GO

In [18]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they are in the External Table */
DROP TABLE IF EXISTS dbo.CustomersWithBadCredit;

CREATE TABLE dbo.CustomersWithBadCredit
(
    CustomerName varchar(100)
);

INSERT INTO dbo.CustomersWithBadCredit
VALUES
    ('Abel Pirvu'),
    ('Adirake Saenamuang'),
    ('Crina Nechita');


--Combine SQL Data with HDFS to show the email address of customers with bad credit.
SELECT hdfs.*
FROM partner_customers_hdfs hdfs
WHERE
    hdfs.CustomerName IN (SELECT CustomerName FROM dbo.CustomersWithBadCredit);


## Next Steps: Continue on to Working with the SQL Server Data Pool

Now you're ready to open the next Python Notebook - `notebook_03.ipynb` - to learn how to create and work with a Data Mart.